In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [ ]:
import pygmt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [ ]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [ ]:
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
# Plot TDE displacements from a single element
# tri_idx = 0
# vel_east, vel_north, vel_up = celeri.get_tri_displacements(station.lon.to_numpy(), station.lat.to_numpy(), meshes, command.material_lambda, command.material_mu, tri_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0)
# plt.figure()
# plt.plot(meshes[0].lon1[tri_idx], meshes[0].lat1[tri_idx], ".r")
# plt.plot(meshes[0].lon2[tri_idx], meshes[0].lat2[tri_idx], ".r")
# plt.plot(meshes[0].lon3[tri_idx], meshes[0].lat3[tri_idx], ".r")
# plt.quiver(station.lon, station.lat, vel_east, vel_north, scale=1e-3)
# plt.show()

In [ ]:
# Plot TDE displacements from a single element
tri_idx = 0
vel_east, vel_north, vel_up = celeri.get_tri_displacements(station.lon.to_numpy(), station.lat.to_numpy(), meshes, command.material_lambda, command.material_mu, tri_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0)
plt.figure()
plt.plot(meshes[0].lon1[tri_idx], meshes[0].lat1[tri_idx], ".r")
plt.plot(meshes[0].lon2[tri_idx], meshes[0].lat2[tri_idx], ".r")
plt.plot(meshes[0].lon3[tri_idx], meshes[0].lat3[tri_idx], ".r")
plt.quiver(station.lon, station.lat, vel_east, vel_north, scale=1e-3)
plt.show()

In [ ]:
# Create a plot with coast, Miller projection (J) over the continental US
min_lon = -135.0
max_lon = -110.0
min_lat = 30.0
max_lat = 50.0
region = [min_lon, max_lon, min_lat, max_lat]
topo_data = '@earth_relief_30s'
# topo_data = '@earth_relief_01m'
projection = "J-65/12c"

fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")
pygmt.config(MAP_FRAME_PEN="0.25p")
pygmt.config(MAP_TICK_PEN="0.25p")

pygmt.makecpt(
    cmap='topo',
    # cmap='terra',
    # series='-4000/4000/20',
    # series='-11000/4000/500',
    continuous=True
)

fig.grdimage(
    grid=topo_data,
    region=region,
    # shading=True,
    projection=projection,
    )

fig.coast(
    region=region,
    projection=projection,
    # land="lightgray",
    # water="white",
    area_thresh=4000,
    shorelines="0.25p,200/200/200",
    frame="p",
)

# Plot block boundaries
for i in range(closure.n_polygons()):
    fig.plot(x=closure.polygons[i].vertices[:, 0], y=closure.polygons[i].vertices[:, 1], pen="0.25p,0/0/0")

# Plot GPS stations
# fig.plot(x=station.lon, y=station.lat, style="c0.05", color="yellow", pen="0.1p,black")
fig.show()

In [ ]:
# Stained glass block polygons
fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")
pygmt.config(MAP_FRAME_PEN="0.25p")
pygmt.config(MAP_TICK_PEN="0.25p")

pygmt.makecpt(
    cmap='gray',
    series='-4000/4000/20',
    continuous=True
)

fig.grdimage(
    grid=topo_data,
    region=region,
    shading=True,
    projection=projection,
    )

# Plot blocks as transparent polygons
for i in range(closure.n_polygons()):
    red = 100 + 50 * np.random.rand(1)
    green = 150 + 100 * np.random.rand(1)
    blue = 150 + 100 * np.random.rand(1)
    color_string = str(red[0]) + "/" + str(green[0]) + "/" + str(blue[0]) + "@75"
    fig.plot(x=closure.polygons[i].vertices[:, 0], y=closure.polygons[i].vertices[:, 1], color=color_string)

# Plot block boundaries
for i in range(closure.n_polygons()):
    fig.plot(x=closure.polygons[i].vertices[:, 0], y=closure.polygons[i].vertices[:, 1], pen="0.1p,0/0/0")
    
fig.coast(
    region=region,
    projection=projection,
    area_thresh=4000,
    shorelines="0.25p,255/255/255",
    frame="p",
)

fig.show()

# Visualization of matrix operators

In [ ]:
#TODO: add generation of matrix operators

# Shrink operators.tri_station so that there are no vertical displacments and no tensile slip
tde_matrix = copy.deepcopy(operators.tri_station)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[0], 3), axis=0)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[1], 3), axis=1)

# Slip estimation with direct inverse and smoothing matrix
smoothing_matrix = 1e2 * meshes[0].smoothing_matrix.toarray()
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[0], 3), axis=0)
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[1], 3), axis=1)
smoothing_matrix = meshes[0].smoothing_weight * smoothing_matrix # Weight smoothing matrix
tde_and_smoothing_matrix = np.vstack((tde_matrix, smoothing_matrix))

print(tde_matrix.shape)
print(smoothing_matrix.shape)
print(tde_and_smoothing_matrix.shape)

# Matrix figures for presentations...should probably replicate elsewhere
# clip_column = 200
# tde_matrix_small = copy.deepcopy(tde_matrix)
# tde_matrix_small = tde_matrix_small[:, 0:clip_column]
# smoothing_matrix_small = copy.deepcopy(smoothing_matrix)
# smoothing_matrix_small += 1e-10
# smoothing_matrix += 1e-10
# smoothing_matrix_small = smoothing_matrix_small[0:clip_column, 0:clip_column]

# plt.figure(figsize=(10,10))
# # plt.imshow(np.log10(np.abs(tde_matrix)), cmap="inferno", interpolation="none")
# plt.imshow(np.log10(np.abs(smoothing_matrix_small)), cmap="inferno_r", interpolation="none")
# plt.gca().set_aspect("equal", adjustable="box")
# plt.xticks([])
# plt.yticks([])
# plt.show()
# plt.savefig("smoothing_matrix_small.png", dpi=300)

# Visualize mesh perimeter

celeri.get_ordered_edge_nodes(meshes)
plt.figure(figsize=(12,7))
plt.plot(meshes[0].meshio_object.points[meshes[0].ordered_edge_nodes[:, 0], 0], meshes[0].meshio_object.points[meshes[0].ordered_edge_nodes[:, 0], 1])